# Fact Orders

In [0]:
df = spark.sql("select * from retail_databricks_catalog.silver_schema.orders_silver")
df.display()

In [0]:
%sql
select * from retail_databricks_catalog.gold_schema.dimproducts

In [0]:
df_dimcus = spark.sql("select DimCustomerKey , customer_id as dim_customer_id from retail_databricks_catalog.gold_schema.dimcustomers")
df_dimcus.display()
df_dimpro = spark.sql("select product_id as DimProductKey , product_id as dim_product_id from retail_databricks_catalog.gold_schema.dimproducts")
df_dimpro.display()